# 問4-15 流列のイミュニゼーション

In [1]:
import pandas as pd
import numpy as np

スポットレートの入力

In [2]:
workSheet = pd.DataFrame(data=[
    [1, 7.670/100],
    [2, 8.270/100],
    [3, 8.810/100],
    [4, 9.310/100],
    [5, 9.750/100],
    [6, 10.16/100],
    [7, 10.52/100],
    [8, 10.85/100],
    [9, 11.15/100],
    [10, 11.42/100],
    [11, 11.67/100],
    [12, 11.89/100]
    ],
    columns=['年', 'スポット'])

In [3]:
workSheet.head()

,年,スポット
0,1,0.0767
1,2,0.0827
2,3,0.0881
3,4,0.0931
4,5,0.0975


割引率の計算
$$ d_{0,k}=(1+f_{0,k})^{-k}=(1+s_k)^{-k}$$

In [4]:
workSheet['d']=(1+workSheet['スポット'])**(-workSheet['年'])

債務の流列$B_0$、債券の流列$B_1$、$B_2$を入力する。

In [5]:
workSheet['B0']=[500,900,600,500,100,100,100,50,0,0,0,0]
workSheet['B1']=[6,6,6,6,6,6,6,6,6,6,6,106]
workSheet['B2']=[10,10,10,10,110,0,0,0,0,0,0,0]

それぞれのキャッシュフローの現在価値$PV$を計算する。($i\in\{0,1,2\}$)
$$PV_{i,k}=B_{i,k}\times d_{0,k}$$
$$PV_i=\sum_{k=1}^nPV_{i,k}$$

In [6]:
workSheet['PV0']=workSheet['B0']*workSheet['d']
workSheet['PV1']=workSheet['B1']*workSheet['d']
workSheet['PV2']=workSheet['B2']*workSheet['d']

In [7]:
PV = workSheet.loc[:,workSheet.columns.str.contains('PV')].sum(axis=0)
PV

PV0    2238.437651
PV1      65.951416
PV2     101.667724
dtype: float64

準修正デュレーション
$$PV'_{i,k}=k\times (1+s_k)^{-(k+1)}B_{i,k}$$
$$D_Q=\frac{1}{PV_i}\sum_{k=1}^nPV'_{i,k}$$

In [8]:
workSheet['PV`0']=workSheet['年']*(1+workSheet['スポット'])**(-(workSheet['年']+1))*workSheet['B0']
workSheet['PV`1']=workSheet['年']*(1+workSheet['スポット'])**(-(workSheet['年']+1))*workSheet['B1']
workSheet['PV`2']=workSheet['年']*(1+workSheet['スポット'])**(-(workSheet['年']+1))*workSheet['B2']

In [9]:
D = workSheet.loc[:,workSheet.columns.str.contains('PV`')].sum(axis=0)
D.index = PV.index
D = D/PV
D.index=['D0','D1','D2']
D

D0    2.447604
D1    7.065795
D2    3.798104
dtype: float64

In [10]:
workSheet

,年,スポット,d,B0,B1,B2,PV0,PV1,PV2,PV`0,PV`1,PV`2
0,1,0.0767,0.928764,500,6,10,464.381908,5.572583,9.287638,431.301112,5.175613,8.626022
1,2,0.0827,0.853068,900,6,10,767.761333,5.118409,8.530681,1418.234660,9.454898,15.758163
2,3,0.0881,0.776236,600,6,10,465.741373,4.657414,7.762356,1284.095322,12.840953,21.401589
3,4,0.0931,0.700423,500,6,10,350.211519,4.202538,7.004230,1281.535155,15.378422,25.630703
4,5,0.0975,0.628026,100,6,110,62.802561,3.768154,69.082817,286.116452,17.166987,314.728097
5,6,0.1016,0.559573,100,6,0,55.957259,3.357436,0.000000,304.778102,18.286686,0.000000
6,7,0.1052,0.496494,100,6,0,49.649384,2.978963,0.000000,314.464069,18.867844,0.000000
7,8,0.1085,0.438646,50,6,0,21.932312,2.631877,0.000000,158.284619,18.994154,0.000000
8,9,0.1115,0.386202,0,6,0,0.000000,2.317213,0.000000,0.000000,18.762862,0.000000
9,10,0.1142,0.339132,0,6,0,0.000000,2.034791,0.000000,0.000000,18.262346,0.000000


ポートフォリオのイミュニゼーションを行う。

債券1と債券2への投資単位をそれぞれ$x_1$、$x_2$とすると、

\begin{align}
PV_0 &= x_1PV_1 + x_2PV_2\\
PV_0\times D_0 &= x_1PV_1D_1 + x_2PV_2D_2\\ 
\\
\left[\begin{matrix} PV_0 \\ PV_0D_0 \end{matrix}\right] &= \left[\begin{matrix} PV_1 & PV_2 \\ PV_1D_1 & PV_2D_2 \end{matrix}
\right]\left[\begin{matrix} x_1 \\ x_2 \end{matrix}\right]\\
\\
\left[\begin{matrix} x_1 \\ x_2 \end{matrix}\right] &= \left[\begin{matrix} PV_1 & PV_2 \\ PV_1D_1 & PV_2D_2 \end{matrix}
\right]^{-1}\left[\begin{matrix} PV_0 \\ PV_0D_0 \end{matrix}\right]
\end{align}


In [11]:
A = np.matrix([
    [PV[1], PV[2]],
    [PV[1]*D[1], PV[2]*D[2]]
    ])
b = np.matrix([
    [PV[0]],
    [PV[0]*D[0]]
    ])

x = A**-1*b
x

matrix([[-14.02731012],
        [ 31.1166465 ]])